# Text generation


This file is based on :https://blog.csdn.net/lujohn3li/article/details/79728799

https://github.com/pranjal52/text_generators

In [0]:
import re
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# import os
# os.getcwd()

In [0]:
target_dir = 'gdrive/My Drive/578_project/neg_mobile'
#saving all the files to the google drive
# os.mkdir(target_dir)
import os
os.chdir(target_dir)

os.listdir()

['sample_neg_mobilephone90.txt',
 'interactive_conditional_samples.py',
 'encoder.py',
 'load_dataset.py',
 'train.py',
 'train_random.py',
 'generate_unconditional_samples.py',
 'sample.py',
 'model.py',
 'accumulate.py',
 'models',
 '__pycache__',
 'checkpoint',
 'samples',
 'sample_neg_mobilephone5.txt',
 'text_generator_400_0.2_400_0.2_400_0.2_100.h5']

In [0]:
text=(open('sample_neg_mobilephone5.txt').read())
text=text.lower()

In [0]:
print(text[:500])

good looking phone, doesnt work properly. the phone is nice, good looking phone. problem is that when you try to see videos or download apps it doesn't work. it is very slow internet wise.it is more like a 2g phone, even with wi-fi it doesn't work.i didn't want to return it because i liked how it looked but i wouldn't have been able to use facebook, ore so. i tried downloading facebook, twitter,whatsapp, and none downloaded in two days, that is when i decided to give it back.youtube videos can't


In [0]:

characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [0]:
n_to_char


{0: '\n',
 1: ' ',
 2: '!',
 3: '"',
 4: '#',
 5: '$',
 6: '%',
 7: '&',
 8: "'",
 9: '(',
 10: ')',
 11: '*',
 12: '+',
 13: ',',
 14: '-',
 15: '.',
 16: '/',
 17: '0',
 18: '1',
 19: '2',
 20: '3',
 21: '4',
 22: '5',
 23: '6',
 24: '7',
 25: '8',
 26: '9',
 27: ':',
 28: ';',
 29: '<',
 30: '=',
 31: '>',
 32: '?',
 33: '@',
 34: '[',
 35: ']',
 36: '^',
 37: '_',
 38: '`',
 39: 'a',
 40: 'b',
 41: 'c',
 42: 'd',
 43: 'e',
 44: 'f',
 45: 'g',
 46: 'h',
 47: 'i',
 48: 'j',
 49: 'k',
 50: 'l',
 51: 'm',
 52: 'n',
 53: 'o',
 54: 'p',
 55: 'q',
 56: 'r',
 57: 's',
 58: 't',
 59: 'u',
 60: 'v',
 61: 'w',
 62: 'x',
 63: 'y',
 64: 'z',
 65: '{',
 66: '}',
 67: '~',
 68: '¨',
 69: '®',
 70: 'µ',
 71: '·',
 72: 'à',
 73: 'á',
 74: 'ã',
 75: 'ç',
 76: 'é',
 77: 'ê',
 78: 'ì',
 79: 'í',
 80: 'î',
 81: 'ñ',
 82: 'ò',
 83: 'ó',
 84: 'ù',
 85: 'ú',
 86: 'þ',
 87: 'ı',
 88: '\u200b',
 89: '’',
 90: '“',
 91: '”',
 92: '…',
 93: '手',
 94: '机',
 95: '的',
 96: '，',
 97: '：'}

In [0]:
X = []
Y = []
length = len(text)
seq_length = 100

for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [0]:

X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [0]:
with open('sample_neg_mobilephone5.txt', "r") as ins:
    out_text = ins.readlines()
out_text = [x.strip().lower() for x in out_text ] 

In [0]:
def generate_smaple(  seq_length = 100,gen_length = 400):
  #generate one sample
  import random
  test_sampleset = random.sample(out_text, 1)
  string_mapped = []
  length = len(text)
  sequence = text[0:seq_length]
  string_mapped = ([char_to_n[char] for char in sequence])
  full_string = [n_to_char[value] for value in string_mapped]
    #combining text
  txt=""
  for char in full_string:
      txt = txt+char
  print(txt)
  # generating characters
  from tqdm import tqdm
  with tqdm(total=gen_length,desc = "generating progress") as pbar:
    for i in range(gen_length):
        x = np.reshape(string_mapped,(1,len(string_mapped), 1))
        x = x / float(len(characters))

        pred_index = np.argmax(model.predict(x, verbose=0))
        seq = [n_to_char[value] for value in string_mapped]
        full_string.append(n_to_char[pred_index])
        string_mapped.append(pred_index)
        string_mapped = string_mapped[1:len(string_mapped)]
        pbar.update(1)

  #combining text
  txt=""
  for char in full_string:
      txt = txt+char
  return(txt)

In [0]:
X_modified.shape

(2648048, 100, 1)

In [0]:
# # A baseline model
# model = Sequential()
# model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(400))
# model.add(Dropout(0.2))
# model.add(Dense(Y_modified.shape[1], activation='softmax'))

# model.compile(loss='categorical_crossentropy', optimizer='adam')



# A deeper model

model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
for i in range(1):
  model.fit(X_modified, Y_modified, epochs =1 , batch_size=1000)
  print(generate_smaple())

# model.save_weights('text_generator_400_0.2_400_0.2_400_0.2_100.h5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
2648048/2648048 [==============================] - 2955s 1ms/step - loss: 1.3913


generating progress:   0%|          | 0/400 [00:00<?, ?it/s]

good looking phone, doesnt work properly. the phone is nice, good looking phone. problem is that whe


generating progress: 100%|██████████| 400/400 [01:01<00:00,  6.83it/s]

good looking phone, doesnt work properly. the phone is nice, good looking phone. problem is that when i try to return it because i was able to get a phone that is the same phone i have to send it back to the phone and i have to send it back to the phone and i have to send it back to the phone and i have to send it back to the phone and i have to send it back to the phone and i have to send it back to the phone and i have to send it back to the phone and i have to send it back to the phone and i 


In [0]:
generate_smaple()
  model.fit(X_modified, Y_modified, epochs =1 , batch_size=100)
  print(generate_smaple())

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
1772400/2648048 [===================>..........] - ETA: 59:15 - loss: 1.5819

In [0]:

model.save_weights('text_generator_400_0.2_400_0.2_400_0.2_100.h5')

# New Section

In [0]:
model.load_weights('text_generator_400_0.2_400_0.2_400_0.2_100.h5')

## Generating Text¶

In [0]:
generate_smaple()

generating progress:   0%|          | 0/400 [00:00<?, ?it/s]

good looking phone, doesnt work properly. the phone is nice, good looking phone. problem is that whe


generating progress: 100%|██████████| 400/400 [01:04<00:00,  6.28it/s]


"good looking phone, doesnt work properly. the phone is nice, good looking phone. problem is that when i couldn't recommend this phone for a seplacement and it seems to be a seal seview i had to return it because it s a seal seview i had to return it because it s a seal seview i had to return it because it s a seal seview i had to return it because it s a seal seview i had to return it because it s a seal seview i had to return it because it s a seal seview i had to return it because it s a seal "